+ Pixelized shear measurements (smoothing -> pixelize/ pixelize -> smoothing)
+ Get the variance map (mock/analytical)

In [2]:
%pylab inline
import os
import gc
import glob
import numpy as np
import astropy.io.fits as pyfits
import maputil as mapUtil
from configparser import ConfigParser
from pixel3D import cartesianGrid3D

Populating the interactive namespace from numpy and matplotlib


In [3]:
key='dempz'
configName  =   'wrkDir/config-nl15.ini'
parser      =   ConfigParser()
parser.read(configName)

configName  =   'wrkDir/config-nl15.ini'
pDict={'dempz': 'demp',\
       'dnnz': 'dnn',\
       'mizuki': 'mizuki'}

In [12]:
colname=pDict[key]
for fieldname in mapUtil.field_names:
    inDir  =   'wrkDir/reGaus_catalog/%s_groups/' %fieldname
    outDir =   'wrkDir/reGaus_pixel3D/%s_groups/' %fieldname
    if not os.path.isdir(outDir):
        os.mkdir(outDir)
    fnames=glob.glob(os.path.join(inDir,'src-group-*.fits'))
    for fname in fnames:
        gn=fname.split('-')[-1].split('.')[0]
        # update zbounds
        bname=os.path.join(outDir,'group-%s-%s-10bins.fits' %(gn,key))
        bounds=pyfits.getdata(bname,ext=1)['zbounds']
        parser.set('sourceZ','zbound','[%s]' %(','.join([str(bb) for bb in bounds])))
        gridInfo    =   cartesianGrid3D(parser)
        glab=fieldname+"-%s"%gn
        # poz average
        data=pyfits.getdata(fname)
        mask=(~np.isnan(data['%s_zbest' %colname]))\
            &(data['%s_zbest_std'%colname]<1.0)\
            &(data['%s_zbest'%colname]<3.0)\
            &(data['%s_zbest'%colname]>0.05)
        data=data[mask]
        wsAve=np.average(data['weight'])
        ws=data['weight']/wsAve
        z=data['%s_zbest'%colname]
        x,y=gridInfo.setupTanPlane(data['ra'],data['dec'])
        out1=gridInfo.pixelize_data(x,y,z,data['g1'],ws=ws)
        out2=gridInfo.pixelize_data(x,y,z,data['g2'],ws=ws)
        outname1=os.path.join(outDir,'group-%s-%s-g1Map.fits' %(gn,key))
        outname2=os.path.join(outDir,'group-%s-%s-g2Map.fits' %(gn,key))
        pyfits.writeto(outname1,out1,overwrite=True)
        pyfits.writeto(outname2,out2,overwrite=True)
        outname3=os.path.join(outDir,'group-%s-%s-g1Hist.png' %(gn,key))
        mapUtil.plotHist(glab,data['g1'],out1[out1!=0],outname3)
        del out1,out2,data,bounds,x,y,z,ws
        gc.collect()

# Mock

In [9]:
nsim=100
colname=pDict[key]
for fieldname in mapUtil.field_names:
    inDir  =   'wrkDir/reGaus_catalog/%s_groups/' %fieldname
    outDir =   'wrkDir/reGaus_pixel3D/%s_groups/' %fieldname
    fnames=glob.glob(os.path.join(inDir,'src-group-*.fits'))
    for fname in fnames:
        gn=fname.split('-')[-1].split('.')[0]
        outDir2=os.path.join(outDir,'mock-group-%s'%gn)
        if not os.path.isdir(outDir2):
            os.mkdir(outDir2)
        # update zbounds
        bname=os.path.join(outDir,'group-%s-%s-10bins.fits' %(gn,key))
        bounds=pyfits.getdata(bname,ext=1)['zbounds']
        parser.set('sourceZ','zbound','[%s]' %(','.join([str(bb) for bb in bounds])))
        gridInfo    =   cartesianGrid3D(parser)
        data=pyfits.getdata(fname)
        mask=(~np.isnan(data['%s_zbest' %colname]))\
            &(data['%s_zbest_std'%colname]<1.0)\
            &(data['%s_zbest'%colname]<3.0)\
            &(data['%s_zbest'%colname]>0.05)
        data=data[mask]
        wsAve=np.average(data['weight'])
        ws=data['weight']/wsAve
        z=data['%s_zbest'%colname]
        x,y=gridInfo.setupTanPlane(data['ra'],data['dec'])
        del data,bounds
        gc.collect()
        for i in range(nsim):
            # poz average
            fname2=os.path.join(inDir,'mock-group-%s'%gn,'sim-%d.fits'%i)
            data=pyfits.getdata(fname2)[mask]
            out1=gridInfo.pixelize_data(x,y,z,data['g1'],ws=ws)
            out2=gridInfo.pixelize_data(x,y,z,data['g2'],ws=ws)
            outname1=os.path.join(outDir2,'sim-%s-%s-g1Map.fits' %(i,key))
            outname2=os.path.join(outDir2,'sim-%s-%s-g2Map.fits' %(i,key))
            pyfits.writeto(outname1,out1,overwrite=True)
            pyfits.writeto(outname2,out2,overwrite=True)
            del out1,out2,data
            gc.collect()
        break
    break

In [5]:
nsim=100
for fieldname in mapUtil.field_names:
    inDir  =   'wrkDir/reGaus_catalog/%s_groups/' %fieldname
    outDir =   'wrkDir/reGaus_pixel3D/%s_groups/' %fieldname
    fnames=glob.glob(os.path.join(inDir,'src-group-*.fits'))
    for fname in fnames:
        gn=fname.split('-')[-1].split('.')[0]
        outDir2=os.path.join(outDir,'mock-group-%s'%gn)
        gStd=np.zeros(pyfits.getdata(os.path.join(outDir2,'sim-0-%s-g1Map.fits' %key)).shape)
        for i in range(nsim):
            gStd+=(pyfits.getdata(os.path.join(outDir2,'sim-%s-%s-g1Map.fits' %(i,key))))**2.
            gStd+=(pyfits.getdata(os.path.join(outDir2,'sim-%s-%s-g2Map.fits' %(i,key))))**2.
        gStd/=nsim
        gStd=np.sqrt(gStd)
        pyfits.writeto(os.path.join(outDir,'group-%s-%s-stdMap.fits' %(gn,key)),gStd,overwrite=True)
        break
    break